In [13]:
from flask import Flask, request, jsonify
import os
import serpapi
import pandas as pd
import tensorflow as tf
from keras.preprocessing import image
import numpy as np

import logging
from PIL import Image
import io

In [2]:
app = Flask(__name__)

logging.basicConfig(level=logging.INFO)
# Load dataset
train_path = "C:/Users/rashe/ImageRecognitionProject/Groceries-Images-Recognition/ML/dataset/train"
image_category = os.listdir(train_path)

# Apply the suitable Image Preprocessing by using ImageDataGenerator
train_generator = ImageDataGenerator(rescale=1.0/255.0)
train_image_generator = train_generator.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Load the trained model
model = tf.keras.models.load_model('C:/Users/rashe/ImageRecognitionProject/Groceries-Images-Recognition/ML/mymodel.h5')

# Define the class map for label mapping
class_map = dict([v, k] for k, v in train_image_generator.class_indices.items())

@app.route('/')
def index():
    return 'Image Recognition API'


@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the image from the POST request
        img_file = request.files['image']
        
        # Check if the file is an image
        if img_file.filename == '':
            return jsonify({"error": "No image provided"}), 400
        
        # Load and preprocess the image
        img = Image.open(io.BytesIO(img_file.read()))
        img = img.resize((224, 224))  # Resize to match the model's input shape
        img_array = img_to_array(img) / 255.0
        img_input = img_array.reshape((1, img_array.shape[0], img_array.shape[1], img_array.shape[2]))

        # Debug: Print image shape
        print("Image shape:", img_input.shape)

        # Make prediction
        predictions = model.predict(img_input)
        print("Raw predictions:", predictions)

        predicted_label = int(predictions.argmax(axis=-1))
        print("Predicted label index:", predicted_label)
        
        # Debug: Print class map
        print("Class map:", class_map)

        predicted_img = class_map[predicted_label]

        # Return the result
        return jsonify({"predicted_product": predicted_img})

    except Exception as e:
        logging.error(f"Error during prediction: {e}")
        return jsonify({"error": "Failed to predict"}), 500

if __name__ == '__main__':
    app.run(debug=False)

NameError: name 'Flask' is not defined